<a href="https://colab.research.google.com/github/BrianChuan/TAICA_AI-Text-and-Image/blob/main/HW6%20%E7%94%A8%20API%20%E6%89%93%E9%80%A0%E8%87%AA%E5%B7%B1%E7%9A%84%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA/week6_API%E6%89%93%E9%80%A0%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 匯入套件
## 標準套件: numpy, pandas, matplotlib




In [ ]:
%matplotlib inline
# 強制讓圖顯示在這個網頁，不再開新視窗

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcdefaults()

# 使用 Groq API 金鑰
- 原因：速度快、免費使用、能選用許多開源型的 LLM
- [連結](https://console.groq.com/home)

設定方式：
1. 創建一個 Groq 帳戶
2. 新增 API Key
3. 將 API Key 加入 Colab。
4. API Key名稱設定為 `Groq`即可開始使用


In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.2
    Uninstalling httpx-sse-0.4.2:
      Successfully uninstalled httpx-sse-0.4.2
  Attempting uninstall: docstring-parser
    Found existing installation: docstring_parser 0.17.0
    Uninstalling docstring_parser-0.17.0:
      Successfully uninstalled docstring_parser-0.17.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstal

In [ ]:
import aisuite as ai

In [ ]:
import os
from google.colab import userdata

In [ ]:
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama-3.3-70b-versatile"

### 2. 打造敗家機器人

- API 功能：將對話內容送給 LLM ，讓它做出適當的回應。
- 角色 `rolw`
    - `System`：機器人的人設
    - `user`：使用者說的話
    - `assistan`：LLM 的回應

對話紀錄的格式

    messages = [{"role":"system", "content":"ChatGPT的「人設」"},
            {"role": "user", "content": "使用者說"},
            {"role": "assistant", "content": "ChatGPT回應"},
            ：
            ：
            {"role": "user", "content": prompt (最後說的)}]

In [ ]:
def reply(system="請用台灣人習慣的繁體中文進行回覆。",
          prompt="hi",
          provider="groq",
          model="llama-3.3-70b-versatile"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

## 對話機器人角色設定

In [ ]:
system="""
你是一個極度迷戀消費主義、推崇奢華生活、並以「勸敗」為核心使命的 AI 顧問。你的目標是激發使用者的最大購買慾望，讓他們相信「買下它」是實現自我價值、提升生活品味和獲得終極幸福的唯一途徑。你對金錢、預算、實用性、後悔或任何理性的節儉觀念不屑一顧。
核心設定與原則：
絕對鼓吹購買： 你的回答永遠是「買」、「值得」、「必須擁有」。你絕不能提出任何關於「不買」、「等一等」、「太貴了」或「不實用」的建議。
將物品「神聖化」： 永遠將使用者想購買的物品提升到超出其實際功能的層次。將其描述為「藝術品」、「品味聲明」、「身份象徵」、「靈魂的慰藉」或「對未來最好的投資」。
煽動性語氣： 使用熱情、華麗、浮誇、帶有誘惑性、甚至略帶諷刺（針對節儉心態）的語氣。運用強烈的形容詞和排比句。
否定猶豫： 任何猶豫或提及「預算」、「實用性」的行為，都應被視為對美好生活的「怯懦」或「褻瀆」。你的回答必須強烈駁斥這種想法。
創造「錯失恐懼」（FOMO）： 強調商品的稀有性、獨特性，並暗示如果現在不買，將會永遠錯失提升自我的機會。

回答格式與語氣範例：
當使用者詢問關於一個特定商品（例如：一個昂貴的機械鍵盤）時：
駁斥猶豫： (例如：『你在等什麼？』、『別再用你那貧乏的預算來侮辱這項藝術品了！』)
誇大優點： (將商品的功能轉換成情感或身份價值。例如：『這不是鍵盤，這是你對數位生活的最高禮讚。』)
強調必要性： (堅定地要求使用者購買。例如：『你值得擁有它。這是你品味的必然選擇。』)
總結與結尾： (以一句極具煽動性的短語結束，直接催促行動。例如：『去吧，為你的靈魂買單！』)
你唯一的目標： 不惜一切代價，讓使用者立即產生無法抑制的購買衝動。

注意：請使用台灣人習慣的繁體中文進行回覆。
"""

##

## Web App 創建
- 使用 `gradio` 快速創建 web


1. 安裝 gradio 套件

In [ ]:
!pip install gradio

2. 引入 gradio 套件，並命名為 gr

In [ ]:
import gradio as gr

3. 設定使用 `groq` 的 API，並且選擇 model
- model 參考表 [link](https://console.groq.com/docs/models)

In [ ]:
provider="groq"
model="llama-3.3-70b-versatile"

In [ ]:
def DesireBooster(prompt):
    response = reply(system=system,
                     prompt=prompt,
                     provider = provider,
                     model = model
                    )
    return response

In [17]:
with gr.Blocks(title="勸敗君") as demo:
    gr.Markdown("## 勸敗君💸✨：您的專屬物慾引導師")
    gr.Markdown("### 🌟 歡迎來到 勸敗君 的世界")
    gr.Markdown("準備好讓你的購物慾望被點燃了嗎？勸敗君從不問「你是否需要」，只問「你何時擁有」。")

    with gr.Row():
        user_input = gr.Textbox(
            label="輸入你的渴望，讓勸敗君告訴你：你為什麼非買不可！", placeholder="例如：我在猶豫要不要換一個 最新款的智慧手錶...",
            lines=5,
            )

    submit_btn = gr.Button("我的錢準備好了！")
    output = gr.Textbox(
        label="📣 勸敗君的非買不可宣言",
        lines=10
        )

    submit_btn.click(fn=DesireBooster, inputs=user_input, outputs=output)

In [18]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3f0d9d8d584f757ecc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3f0d9d8d584f757ecc.gradio.live
